In [1]:
import numpy as np
import geopandas as gpd
import matplotlib
from osgeo import gdal, ogr
import sys
from matplotlib import pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'geopandas'

Load some data

In [2]:
import os
import gdal
import scipy.ndimage

from shapely.geometry import shape, mapping
from shapely.geometry.multipolygon import MultiPolygon

src_ds = gdal.Open('/Users/rohankumar/Desktop/GeoPlanetary_Project/pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-dtm-v1/mexhrs_2001/data/0360/h0360_0000_re4.img')
srcband = src_ds.GetRasterBand(1)

# Invert pixel value to use srcband as a mask.
numpy_band = srcband.ReadAsArray() ##
numpy_band[numpy_band<125] = 125   ##
numpy_band[numpy_band>125] = 0     ##
srcband.WriteArray(numpy_band)     ##
numpy_band

array([[125, 125, 125, ..., 125, 125, 125],
       [125, 125, 125, ..., 125, 125, 125],
       [125, 125, 125, ..., 125, 125, 125],
       ...,
       [125, 125, 125, ..., 125, 125, 125],
       [125, 125, 125, ..., 125, 125, 125],
       [125, 125, 125, ..., 125, 125, 125]], dtype=uint8)

In [3]:
src_ds.GetProjection()

'PROJCS["SINUSOIDAL MARS",GEOGCS["GCS_MARS",DATUM["D_MARS",SPHEROID["MARS",3396000,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",285],PARAMETER["false_easting",0],PARAMETER["false_northing",0]]'

In [4]:
dst_layername = "polygonized_Stuff"
drv = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = drv.CreateDataSource( dst_layername + ".shp" )
dst_layer = dst_ds.CreateLayer(dst_layername, srs = None )


gdal.Polygonize( srcband, srcband, dst_layer, -1, [], callback=None )  ##
dst_ds.Destroy()
src_ds=None

Changing the projection

In [5]:
world = geopandas.read_file(geopandas.datasets.get_path('polygonized_Stuff'))

NameError: name 'geopandas' is not defined